In [1]:
import pandas as pd
import os

In [49]:
dow_jones_path = os.path.join('Resources', 'DowJones.csv')
nasdaq_path = os.path.join('Resources', 'Nasdaq.csv')
sp500_path = os.path.join('Resources', 'SP500.csv')

In [50]:
dow_jones_df = pd.read_csv(dow_jones_path)
nasdaq_df = pd.read_csv(nasdaq_path)
sp500_df = pd.read_csv(sp500_path)

In [51]:
dow_jones_df = dow_jones_df.rename(columns = {'Value': 'Dow_Jones'})
nasdaq_df = nasdaq_df.rename(columns = {'Value': 'NASDAQ'})
sp500_df = sp500_df.rename(columns = {'Value': 'S&P_500'})

In [52]:
combined_df = sp500_df.merge(nasdaq_df, how = 'outer', on = 'Date')\
              .merge(dow_jones_df, how = 'outer', on = 'Date')

In [53]:
combined_df[combined_df['Dow_Jones'].isna() | combined_df['NASDAQ'].isna() | combined_df['S&P_500'].isna()]

,Date,S&P_500,NASDAQ,Dow_Jones
522,1987-09-04,316.70,NaN,2561.38
1151,1999-09-24,1277.36,NaN,NaN
1254,2001-09-11,1092.54,1695.37,NaN
1346,2003-06-19,994.70,NaN,NaN
1388,2004-04-08,1139.32,2052.88,NaN
1649,2009-04-09,856.56,NaN,NaN
2084,1987-09-4,NaN,446.50,NaN
2085,1999-09-23,NaN,2749.83,10318.59
2086,2003-06-20,NaN,1644.72,9200.75
2087,2001-09-10,NaN,NaN,9605.51
